In [1]:
import pandas as pd
import requests
from pprint import pprint
from splinter import Browser
import time
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
#salary data for NHL data
salary_url = 'https://query.data.world/s/fqj3jtoxu3j6cxluz2umg77haeqcms'
excel_source_df = pd.read_excel(salary_url)
salary_df = excel_source_df.rename(columns={'Tm':'Abbr'})
salary_df = salary_df.replace(to_replace='VEG',value='VGK')
salary_df.head()

,Player,Abbr,Salary,Cap Hit
0,Mitch Marner,TOR,16000000,10893000
1,John Tavares,TOR,15900000,11000000
2,Auston Matthews,TOR,15900000,11634000
3,Carey Price,MTL,15000000,10500000
4,Connor McDavid,EDM,15000000,12500000


In [3]:
#Get the team data as a DataFrame
teams_url = 'https://statsapi.web.nhl.com/api/v1/teams'
json_source_df = pd.read_json(teams_url)

#Count the number of rows (i.e. teams)
team_count = json_source_df['copyright'].count()

#List of row indices for use later
counts = list(range(0, team_count))

#Data presents with nested dictionaries an needs to be cleaned
#Use a for loop to drill down to teams section of JSON data
teams_list = []
for count in counts:
    teams_list.append(json_source_df['teams'][count])

#Cleaning data of undesirable columns
teams_df = pd.DataFrame(teams_list)
teams_df = teams_df.drop(['id','link','venue','officialSiteUrl','franchise','division','conference','shortName'], axis=1)

#Extract desired elements from nested dictionaries
key_list = ['venue','division','conference']
venue_list = []
division_list = []
conference_list = []
#Use for loop to construct lists of desired data
for count in counts:
    venue_list.append(teams_list[count]['venue']['name'])
    division_list.append(teams_list[count]['division']['name'])
    conference_list.append(teams_list[count]['conference']['name'])

#Zip the desired data to a separate DataFrame
venue_df = pd.DataFrame(list(zip(venue_list,division_list,conference_list)),\
                        columns=['venue','division','conference'])

#Join the deisred data into clean data frame
teams_df = teams_df.join(venue_df,how='left')

#scrub column names
teams_df = teams_df.rename(columns={'name':'Name','abbreviation':'Abbr','teamName':'Team','locationName':'Location',\
                                    'firstYearOfPlay':'First Year','franchiseId':'Franchise ID','active':'Active',\
                                    'venue':'Arena','division':'Division','conference':'Conference'
                                   })

#sort as desired (oldest to newest)
teams_df = teams_df.sort_values(by=['First Year','Franchise ID'])

#print it
teams_df


,Name,Abbr,Team,Location,First Year,Franchise ID,Active,Arena,Division,Conference
7,Montréal Canadiens,MTL,Canadiens,Montréal,1909,1,True,Bell Centre,Scotia North,Eastern
9,Toronto Maple Leafs,TOR,Maple Leafs,Toronto,1917,5,True,Scotiabank Arena,Scotia North,Eastern
5,Boston Bruins,BOS,Bruins,Boston,1924,6,True,TD Garden,MassMutual East,Eastern
2,New York Rangers,NYR,Rangers,New York,1926,10,True,Madison Square Garden,MassMutual East,Eastern
14,Chicago Blackhawks,CHI,Blackhawks,Chicago,1926,11,True,United Center,Discover Central,Western
15,Detroit Red Wings,DET,Red Wings,Detroit,1926,12,True,Little Caesars Arena,Discover Central,Eastern
24,Los Angeles Kings,LAK,Kings,Los Angeles,1967,14,True,STAPLES Center,Honda West,Western
23,Dallas Stars,DAL,Stars,Dallas,1967,15,True,American Airlines Center,Discover Central,Western
3,Philadelphia Flyers,PHI,Flyers,Philadelphia,1967,16,True,Wells Fargo Center,MassMutual East,Eastern
4,Pittsburgh Penguins,PIT,Penguins,Pittsburgh,1967,17,True,PPG Paints Arena,MassMutual East,Eastern


In [4]:
players_by_team = salary_df.groupby(['Abbr']).count()
players_by_team = players_by_team.drop(columns=['Salary','Cap Hit'], axis=1)
players_by_team = players_by_team.rename(columns={'Abbr':'Team', 'Player':'Player Count'})
players_by_team

,Player Count
Abbr,
ANA,23
ARI,23
BOS,24
BUF,22
CAR,22
CBJ,26
CGY,23
CHI,22
COL,22


In [5]:
total_players_count = int(players_by_team['Player Count'].sum())
total_players_count

715

In [6]:
team_cap_df = salary_df.drop(['Player','Salary'], axis=1)
team_cap_df = team_cap_df.groupby(['Abbr']).sum()
team_cap_df = team_cap_df.sort_values(by='Cap Hit', ascending=False)
top_cap_df = team_cap_df.iloc[:10]
top_cap_df = top_cap_df.rename(columns={'Cap Hit':'Salary Cap'})
top_cap_df

,Salary Cap
Abbr,
VGK,78910000.0
NSH,78882143.0
NYI,78477499.0
PHI,77335912.0
MIN,76553588.0
FLA,75556249.0
DAL,75041665.0
ARI,74121624.0
STL,74099999.0


In [7]:
top_cap_df2 = top_cap_df.merge(salary_df.drop_duplicates(['Abbr']), how='left', on='Abbr')
top_cap_df2 = top_cap_df2.drop(['Cap Hit'],axis=1)
top_cap_df2

,Abbr,Salary Cap,Player,Salary
0,VGK,78910000.0,Mark Stone,12000000
1,NSH,78882143.0,Matt Duchene,10000000
2,NYI,78477499.0,Anders Lee,9000000
3,PHI,77335912.0,Jakub Voracek,9250000
4,MIN,76553588.0,Ryan Suter,9000000
5,FLA,75556249.0,Sergei Bobrovsky,11500000
6,DAL,75041665.0,Tyler Seguin,13500000
7,ARI,74121624.0,Oliver Ekman-Larsson,8000000
8,STL,74099999.0,Alex Pietrangelo,7500000
9,CGY,72949149.0,Sean Monahan,6750000


In [8]:
top_cap_df3 = top_cap_df2.merge(teams_df, how='left', on='Abbr')
top_cap_df3 = top_cap_df3.drop(columns=['Location','First Year','Franchise ID','Active','Arena',\
                                        'Division','Conference'], axis=1)
top_cap_df3

,Abbr,Salary Cap,Player,Salary,Name,Team
0,VGK,78910000.0,Mark Stone,12000000,Vegas Golden Knights,Golden Knights
1,NSH,78882143.0,Matt Duchene,10000000,Nashville Predators,Predators
2,NYI,78477499.0,Anders Lee,9000000,New York Islanders,Islanders
3,PHI,77335912.0,Jakub Voracek,9250000,Philadelphia Flyers,Flyers
4,MIN,76553588.0,Ryan Suter,9000000,Minnesota Wild,Wild
5,FLA,75556249.0,Sergei Bobrovsky,11500000,Florida Panthers,Panthers
6,DAL,75041665.0,Tyler Seguin,13500000,Dallas Stars,Stars
7,ARI,74121624.0,Oliver Ekman-Larsson,8000000,Arizona Coyotes,Coyotes
8,STL,74099999.0,Alex Pietrangelo,7500000,St. Louis Blues,Blues
9,CGY,72949149.0,Sean Monahan,6750000,Calgary Flames,Flames


In [9]:
### Scrape NHL team logos


img_list = []
#team_list = []

for i in range(len(top_cap_df3)):
    # Set up Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    #URL definitions
    base_url = 'https://www.nhl.com/'
    target_url = base_url + top_cap_df3['Team'][i].lower().replace(' ','')
    browser.visit(target_url)
    #time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    soup1 = soup.find('a', class_='top-nav__club-logo-link')
    scrape = soup1.find_all('img')[0]['src']
    img_link = scrape.replace('https:','')
    img_link = img_link.replace('http:','')
    img_link = img_link.replace('//','')
    #print(img_link)
    img_list.append(img_link)
    browser.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======


...

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/timweir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [10]:
pprint(img_list)

['cms.nhl.bamgrid.com/images/assets/binary/290581542/binary-file/file.svg',
 'www-league.nhlstatic.com/nhl.com/builds/site-core/7250e3e3ba681d881bc3aae25ad6da243c953cda_1617745152/images/logos/team/current/team-18-dark.svg',
 'cms.nhl.bamgrid.com/images/assets/binary/316482732/binary-file/file.svg',
 'www-league.nhlstatic.com/images/logos/teams-current-primary-light/4.svg',
 'cms.nhl.bamgrid.com/images/assets/binary/302317224/binary-file/file.svg',
 'cms.nhl.bamgrid.com/images/assets/binary/291015530/binary-file/file.svg',
 'www-league.nhlstatic.com/images/logos/teams-current-primary-light/25.svg',
 'cms.nhl.bamgrid.com/images/assets/binary/309994320/binary-file/file.svg',
 'cms.nhl.bamgrid.com/images/assets/binary/309991890/binary-file/file.svg',
 'cms.nhl.bamgrid.com/images/assets/binary/319279446/binary-file/file.svg']
